<a href="https://colab.research.google.com/github/Riju0045/ISI-codes/blob/main/ISI_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df1 = pd.read_excel("/content/final data for Ideas proj.xlsx", sheet_name="Sheet 1")  # Order-level
df2 = pd.read_excel("/content/final data for Ideas proj.xlsx", sheet_name="Sheet 3")  # Product-level

In [ ]:
#monthly trend revenue
months = [
    "January", "February", "March", "April", "May", "June", "July",
    "August", "September", "October", "November", "December"
]
monthly_revenue = df2.groupby("Month")["Lineitem price"].sum().reindex(months).dropna()

plt.figure(figsize=(8, 5))
monthly_revenue.plot(marker='o')
plt.title("Monthly Revenue Trend")
plt.xlabel("Month")
plt.ylabel("Revenue (INR)")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
hourly_avg_sales = df1.groupby(['Day', 'Hour'])['Total'].mean().reset_index()

heatmap_data = hourly_avg_sales.pivot(index='Hour', columns='Day', values='Total')

plt.figure(figsize=(16, 8))
sns.heatmap(heatmap_data, cmap='YlOrRd', annot=True, fmt=".0f", linewidths=0.5)

plt.title('Average Sales Amount per Order by Hour and Day of Month', fontsize=14)
plt.xlabel('Day of Month')
plt.ylabel('Hour of Day')
plt.tight_layout()
plt.show()

In [ ]:
#Group by Shipping Method and calculate average order value
aov_shipping = df1.groupby("Shipping Method")["Total"].mean().dropna()

plt.figure(figsize=(8, 5))
ax = aov_shipping.plot(kind="bar", color="red")

plt.title("Average Order Value by Shipping Method")
plt.ylabel("Average Order Value (INR)")
plt.xlabel("Shipping Method")

plt.xticks(rotation=90, fontsize=8)

for i, value in enumerate(aov_shipping):
    ax.text(i, value + 0.01 * max(aov_shipping), f"{value:,.0f}",
            ha='center', va='bottom', fontsize=8, rotation=90)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

df2["Vendor"] = df2["Vendor"].astype(str).str.strip().str.title()

vendor_sales = df2.groupby("Vendor")["Lineitem price"].sum().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
ax = vendor_sales.plot(kind="bar", color="salmon")

plt.title("Total Sales by Vendor")
plt.ylabel("Revenue (INR)")
plt.xlabel("Vendor")
plt.xticks(rotation=45)
plt.tight_layout()

for i, value in enumerate(vendor_sales):
    ax.text(i, value + 0.01 * max(vendor_sales), f"{value:,.0f}", ha='center', va='bottom', fontsize=9, rotation=90)

plt.show()


In [ ]:
df1["Vendor"] = df1["Vendor"].astype(str).str.strip().str.title()
avg_sales_per_order = df1.groupby('Vendor')['Total'].mean().sort_values(ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(x=avg_sales_per_order.index, y=avg_sales_per_order.values, color='coral')

for i, v in enumerate(avg_sales_per_order.values):
    plt.text(i, v + max(avg_sales_per_order.values) * 0.01, f"{v:,.2f}",
             ha='center', va='bottom', fontsize=9)

plt.xticks(rotation=90, ha='right', fontsize=10)
plt.ylabel("Average Per-Order Sales (INR)")
plt.title("Average Sales per Order by Vendor")
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

df_orders = pd.read_excel("/content/final data for Ideas proj.xlsx", sheet_name="Sheet 1")
df_items = pd.read_excel("/content/final data for Ideas proj.xlsx", sheet_name="Sheet 3")


In [ ]:
df_orders["Day"] = pd.to_datetime(df_orders["Day"], errors='coerce')
latest_date = df_orders["Day"].max()

rfm = df_orders.groupby("Billing name").agg({
    "Day": lambda x: (latest_date - x.max()).days,  # Recency in days
    "Name": "count",                                # Frequency
    "Subtotal": "sum"                               # Monetary
}).rename(columns={"Day": "Recency", "Name": "Frequency", "Subtotal": "Monetary"}).dropna()

scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm)

kmeans_rfm = KMeans(n_clusters=4, random_state=42)
rfm["RFM_Cluster"] = kmeans_rfm.fit_predict(rfm_scaled)
rfm.groupby("RFM_Cluster").mean()

In [ ]:
rfm.reset_index(inplace=True)

rfm[["Billing name", "Recency", "Frequency", "Monetary", "RFM_Cluster"]].to_csv("rfm_customer_clusters.csv", index=False)

print("Customer-cluster mapping exported to 'rfm_customer_clusters.csv'")summary = rfm.groupby("RFM_Cluster")[["Recency", "Frequency", "Monetary"]].agg(["mean", "count"]).round(2)
print(summary)


In [ ]:
# Order Pattern Clustering
df_orders["Hour"] = df_orders["Hour"].astype(int)

pattern_data = df_orders[["Total", "Discount Amount", "Hour", "Total"]].dropna() # Removed 'Day' column
pattern_scaled = scaler.fit_transform(pattern_data)

kmeans_pattern = KMeans(n_clusters=3, random_state=42)
df_orders["OrderCluster"] = kmeans_pattern.fit_predict(pattern_scaled)

In [ ]:
df_order_clusters = df_orders[["Billing name", "OrderCluster"]].drop_duplicates()
df_order_clusters.to_csv("order_pattern_clusters.csv", index=False)

In [ ]:
#Discount Sensitivity Clustering
discount_data = df_orders[["Discount Amount", "Subtotal", "Total"]].dropna()
discount_scaled = scaler.fit_transform(discount_data)

kmeans_discount = KMeans(n_clusters=3, random_state=42)
df_orders["DiscountCluster"] = kmeans_discount.fit_predict(discount_scaled)

In [ ]:
df_discount_clusters = df_orders[["Billing name", "DiscountCluster"]].drop_duplicates()
df_discount_clusters.to_csv("discount_sensitivity_clusters.csv", index=False)

In [ ]:
#Multi-Channel Clustering
if "Source" in df_orders.columns:
    channel_data = df_orders[["Source", "Total", "Discount Amount", "Total"]].dropna()
    channel_data = pd.get_dummies(channel_data, columns=["Source"])
    channel_scaled = scaler.fit_transform(channel_data)

    kmeans_channel = KMeans(n_clusters=3, random_state=42)
    df_orders["ChannelCluster"] = kmeans_channel.fit_predict(channel_scaled)

In [ ]:
if "ChannelCluster" in df_orders.columns:
    df_channel_clusters = df_orders[["Billing name", "ChannelCluster"]].drop_duplicates()
    df_channel_clusters.to_csv("multi_channel_clusters.csv", index=False)

In [ ]:
#Visualize RFM Clusters
pca = PCA(n_components=2)
rfm_pca = pca.fit_transform(rfm_scaled)
rfm["PC1"], rfm["PC2"] = rfm_pca[:, 0], rfm_pca[:, 1]

plt.figure(figsize=(8, 6))
sns.scatterplot(data=rfm, x="PC1", y="PC2", hue="RFM_Cluster", palette="Set2")
plt.title("RFM Clusters (Customer Segmentation)")
plt.tight_layout()
plt.show()

# -----------------------------------
# SAVE OUTPUTS (Optional)
# -----------------------------------
rfm.to_csv("rfm_clusters.csv")
df_orders.to_csv("orders_with_clusters.csv", index=False)


In [ ]:
pca = PCA(n_components=2)
pattern_pca = pca.fit_transform(pattern_scaled)
df_orders["PC1"], df_orders["PC2"] = pattern_pca[:, 0], pattern_pca[:, 1]

plt.figure(figsize=(8, 6))
sns.scatterplot(data=df_orders, x="PC1", y="PC2", hue="OrderCluster", palette="Set1", s=100)
plt.title("Order Pattern Clusters (PCA)")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
pca = PCA(n_components=2)
discount_pca = pca.fit_transform(discount_scaled)
df_orders["PC1_disc"], df_orders["PC2_disc"] = discount_pca[:, 0], discount_pca[:, 1]

plt.figure(figsize=(8, 6))
sns.scatterplot(data=df_orders, x="PC1_disc", y="PC2_disc", hue="DiscountCluster", palette="cool", s=100)
plt.title("Discount Sensitivity Clusters (PCA)")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
if "ChannelCluster" in df_orders.columns:
    pca = PCA(n_components=2)
    channel_pca = pca.fit_transform(channel_scaled)
    df_orders["PC1_chan"], df_orders["PC2_chan"] = channel_pca[:, 0], channel_pca[:, 1]

    plt.figure(figsize=(8, 6))
    sns.scatterplot(data=df_orders, x="PC1_chan", y="PC2_chan", hue="ChannelCluster", palette="Set3", s=100)
    plt.title("Channel-Based Clusters (PCA)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()